In [3]:
import geopandas as gpd
from pathlib import Path

In [10]:
test_data = gpd.read_file("/app/data/test_data.zip")

In [ ]:
from pyproj import Transformer

# Define coordinates in EPSG:4326 (WGS84)
wgs_coords = {
    "minx": 20.6455928891,
    "maxx": 31.5160921567,
    "miny": 59.846373196,
    "maxy": 70.1641930203,
}

# Create a transformer from EPSG:4326 to EPSG:3067
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3067", always_xy=True)

# Transform each corner point
sw = transformer.transform(wgs_coords["minx"], wgs_coords["miny"])
se = transformer.transform(wgs_coords["maxx"], wgs_coords["miny"])
ne = transformer.transform(wgs_coords["maxx"], wgs_coords["maxy"])
nw = transformer.transform(wgs_coords["minx"], wgs_coords["maxy"])

# Get the bounding box in ETRS89/TM35FIN coordinates
etrs_coords = {
    "minx": min(sw[0], nw[0]),
    "maxx": max(se[0], ne[0]),
    "miny": min(sw[1], se[1]),
    "maxy": max(nw[1], ne[1]),
}

print("EPSG:3067 coordinates (ETRS89/TM35FIN):")
print(f"minx: {etrs_coords['minx']}")
print(f"maxx: {etrs_coords['maxx']}")
print(f"miny: {etrs_coords['miny']}")
print(f"maxy: {etrs_coords['maxy']}")

EPSG:3067 coordinates (ETRS89/TM35FIN):
minx: 144286.33218675363
maxx: 752934.2155768903
miny: 6642928.395443255
maxy: 7796732.440183549
